# Basic Model


# Importing Necessary Modules
 

In [1]:
from paddleocr import PaddleOCR, draw_ocr 
from matplotlib import pyplot as plt 
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import logging
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
logging.getLogger("ppocr").setLevel(logging.WARNING)
logging.getLogger("ppocr").setLevel(logging.CRITICAL)
from telegram.ext import *
from selenium.common.exceptions import *
from datetime import *
import re
from translate import Translator

drug_file = 'D:/Projctzz/DHANVANTRI/drugs.csv'                  # give path of the drug file 
Token = '6192929005:AAGHaS6NUx2XEpppi1_zksxCgYIzfKR2ZHY'         # update Token id with yours
updater = Updater(token=Token, use_context=True)
dispatcher = updater.dispatcher


c:\Users\DRUVA\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:43: DeprecationWarning: 'urllib3[secure]' extra is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  import urllib3


# OCR Operation


In [2]:
def drug_name(img,drug_file):
    df = pd.read_csv(drug_file,encoding='iso-8859-1')   
    ocr_model = PaddleOCR(lang='en')   
    result = ocr_model.ocr(img)
    element = 1
    drug = ''
    for item in result:
        while True:
            try:
                text_elements = item[element][1][0].replace(',',' ').split()
                for word in text_elements:
                    word = word.upper()
                    if word in df['Drug_Name'].values and word not in drug:
                        drug = drug+' '+word
            except IndexError:
                break
            element+=1
    
    return drug

# Scrapping Work

In [3]:
def drug_det_scrape(drug):
    
    url = 'https://pharmeasy.in'
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    Search_box_click = driver.find_element('xpath','//*[@id="__next"]/main/div[3]/div[1]/div/div[1]/div/div[2]/div/div[1]').click()
    Search_box_text = driver.find_element('xpath','//*[@id="topBarInput"]').send_keys(drug)
    Search = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/main/div[3]/div[1]/div/div[1]/div/div[2]/div/div[1]/div/div/div[2]/a[1]/div/div'))).click()
    product = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/main/div/div/div/div[1]/div[1]/div/div/a/div[1]'))).click()


    best_price_element = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[1]/td[2]')))
    best_price = driver.find_element('xpath','//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[1]/td[2]').text


    uses_element = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[4]/td[2]')))
    uses = driver.find_element('xpath','//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[4]/td[2]').text


    side_effects_element = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[5]/td[2]')))
    side_effects = driver.find_element('xpath','//*[@id="__next"]/main/div/div[2]/div[2]/div[5]/div[2]/table/tbody/tr[5]/td[2]').text

    return best_price,uses,side_effects

# Telegram Bot 

#### Getting Image form Telegram

In [4]:
lang = None
def handle_image(update, context):
    global lang
    if lang == None:
        lang = 'en'
    translator = Translator(to_lang=lang)
    message = update.message
    # global photo_path
    if message.photo:

        # Get the file ID of the largest photo
        file_id = message.photo[-1].file_id

        # Get the file object
        file_obj = context.bot.get_file(file_id)

        # Download the photo
        photo_path = 'received_photo.jpg'
        file_obj.download(photo_path)

        # Respond with a message
        message.reply_text(translator.translate('Photo received!'))

        drug = drug_name(str(photo_path),drug_file)

        if drug == '':
            update.message.reply_text(translator.translate("Name not Detected!\nMake sure that Drug name visible\n "))
            update.message.reply_text("No Worries! Go with /manual_entry or /Features of Dhanvantri")
            return
        update.message.reply_text(translator.translate("Ur results getting Ready Please Wait...."))
        best_price,uses,side_effects = drug_det_scrape(drug)
        update.message.reply_text(translator.translate("Almost There ..."))
        # update.message.reply_text(
        # f'''{translator.translate('USES')}: \n\n {translator.translate(uses)} \n\n {translator.translate('BEST PRICE')} :\n\n {translator.translate(best_price)} \n\n {translator.translate('SIDE EFFECTS')}:\n\n {translator.translate(side_effects)}
        # '''
        update.message.reply_text(Translator.translate(
        f''' USES: \n {uses} \n\n BEST PRICE:\n {best_price} \n\n SIDE EFFECTS:\n {side_effects}''')
        )
        # cannot able to return photo_path
        # if we pass photo directly that will be great

    else:
        # If the message does not contain a photo
        message.reply_text(translator.translate('Please send an image.'))

#### Start

In [5]:
def start(update, context):
    update.message.reply_text("Hi! I'm Dhanvantri The Drug Detector Bot.\nI'm a bot that lets you find drugs in images.\n\nTo /Features, click the Features button to start.") 

#### Help

In [6]:
def help(update,context):
    update.message.reply_text(
        '''
        /start -> Greating From Dhanvanthri\n/help  -> Need any Help\n/Features -> Know What Dhanvanthri can do\n/Change_Language -> Change the language of the Bot\n/identify_drug -> medicine identification\n/remainder -> medicine intake remainder\n/manual_entry -> Manual entry of Drug Name
        '''
    ) 

#### Features

In [7]:
def Features(update,context):
    update.message.reply_text(
        '''
        /Change_Language(default:English)\n/identify_drug\n/remainder\n/manual_entry
        '''
        )

#### Transalate

In [8]:
def transalate(update,context):
    global lang
    lang = update.message.text
    translator = Translator(to_lang=lang)
    update.message.reply_text(translator.translate('Please Upload medicine strip (Backside where Drug name present)'))
    dispatcher.add_handler(MessageHandler(Filters.photo, handle_image))

In [9]:
def select_lang(update,context):
    update.message.reply_text('''
as: Assamese
bn: Bengali
gu: Gujarati
hi: Hindi
kn: Kannada
ml: Malayalam
mr: Marathi
ne: Nepali
or: Oriya
pa: Punjabi
ta: Tamil
te: Telugu
ur: Urdu
en: English
------------------
ar: Arabic
de: German
es: Spanish
fr: French
it: Italian
ja: Japanese
ko: Korean
pt: Portuguese
ru: Russian
zh-CN: Chinese (Simplified)  ''')
    update.message.reply_text("Enter Requried Language Code 🫡")
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, transalate))

#### Identify Drug

In [10]:
def identify_drug(update,context):
    update.message.reply_text('Please Upload medicine strip (Backside where Drug name present)')
    dispatcher.add_handler(MessageHandler(Filters.photo, handle_image))

#### Manual Entry

In [11]:
def manual_search(update,context):
    message = update.message
    if message.text:
        drug = message.text
        update.message.reply_text("Ur results getting Ready Please Wait....")
        best_price,uses,side_effects = drug_det_scrape(drug)
        update.message.reply_text(Translator.translate("Almost There ..."))
        update.message.reply_text(Translator.translate(
        f''' USES: \n {uses} \n\n BEST PRICE:\n {best_price} \n\n SIDE EFFECTS:\n {side_effects}''')
        )

In [12]:
def manual_entry(update,context):
    update.message.reply_text('Enter Drug Name')
    dispatcher.add_handler(MessageHandler(Filters.text & (~Filters.command),manual_search))

#### Remainder

In [13]:
def alarm(update,context):
    update.message.reply_text("Your Remainder Set ✌️")
    message = update.message
    # Access the text of the last message
    last_message = message.text
    time_pattern = r"^(0[1-9]|1[0-2]):[0-5][0-9] (AM|PM)$"
    if re.match(time_pattern,last_message):
        while True:
            current_time = datetime.now().time().strftime("%I:%M %p")
            if current_time == last_message: 
                update.message.reply_text("Time to take medicine 💊 ")
                return
    else:
        update.message.reply_text("Enter Time in Correct order 👀") 
        update.message.reply_text('No Worries 😎 \n Try again -> /remainder')        

In [14]:
def remainder(update,context):
        update.message.reply_text("Enter Alerm Time (HH:MM AM)")
        dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, alarm))

#### Error Handler

In [15]:
def error_handler(update, context):
    # Log the exception
    context.error(f"An error occurred: {context.error}")

#### Running the Bot

In [16]:
dispatcher.add_handler(CommandHandler('Start',start))
dispatcher.add_handler(CommandHandler('Help',help))
dispatcher.add_handler(CommandHandler('Change_Language',select_lang))
dispatcher.add_handler(CommandHandler('Features',Features))
dispatcher.add_handler(CommandHandler('Identify_drug',identify_drug))
dispatcher.add_handler(CommandHandler('Remainder',remainder))
dispatcher.add_handler(CommandHandler('Manual_Entry',manual_entry))
dispatcher.add_error_handler(error_handler)
updater.start_polling()
updater.idle()

[2023-08-23 22:31:18,012] [   ERROR] dispatcher.py:573 - An uncaught error was raised while handling the error.
Traceback (most recent call last):
  File "c:\Users\DRUVA\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "c:\Users\DRUVA\AppData\Local\Programs\Python\Python39\lib\site-packages\telegram\ext\handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "C:\Users\DRUVA\AppData\Local\Temp\ipykernel_8544\199061598.py", line 31, in handle_image
    best_price,uses,side_effects = drug_det_scrape(drug)
  File "C:\Users\DRUVA\AppData\Local\Temp\ipykernel_8544\2435440006.py", line 6, in drug_det_scrape
    driver = webdriver.Chrome(ChromeDriverManager().install())
  File "c:\Users\DRUVA\AppData\Local\Programs\Python\Python39\lib\site-packages\webdriver_manager\chrome.py", line 39, in install
    driver_path = self._get_driver_pat